In [1]:
%load_ext autoreload
%autoreload 2
import os
PATH = os.path.dirname(os.getcwd())
import pandas as pd
import sys
sys.path.append(PATH)
from student_writing.scripts.spacy_model import *

In [2]:
df = pd.read_csv('../student_writing/data/train.csv')

In [8]:
df.head(20)

,id,discourse_start,discourse_end,discourse_text,discourse_type
0,423A1CA112E2,8.0,229.0,Modern humans today are always on their phone....,LEAD
1,423A1CA112E2,230.0,312.0,They are some really bad consequences when stu...,POSITION
2,423A1CA112E2,313.0,401.0,Some certain areas in the United States ban ph...,EVIDENCE
3,423A1CA112E2,402.0,758.0,"When people have phones, they know about certa...",EVIDENCE
4,423A1CA112E2,759.0,886.0,Driving is one of the way how to get around. P...,CLAIM
5,423A1CA112E2,887.0,1150.0,That's why there's a thing that's called no te...,EVIDENCE
6,423A1CA112E2,1151.0,1533.0,Sometimes on the news there is either an accid...,EVIDENCE
7,423A1CA112E2,1534.0,1602.0,Phones are fine to use and it's also the best ...,CLAIM
8,423A1CA112E2,1603.0,1890.0,If you go through a problem and you can't find...,EVIDENCE
9,423A1CA112E2,1891.0,2027.0,The news always updated when people do somethi...,CONCLUDING STATEMENT


## Change to spacy format

In [3]:
X_train_df, X_val_df = split_df(df, val_split=0.3)


In [4]:
X_train = data_to_spacy_format(X_train_df)
X_val = data_to_spacy_format(X_val_df)


100%|██████████| 4678/4678 [00:52<00:00, 89.38it/s] 


In [5]:
spacy_train = make_spacy_model(X_train)
spacy_train.to_disk('../student_writing/models/train.spacy')
spacy_val = make_spacy_model(X_val)
spacy_val.to_disk('../student_writing/models/val.spacy')

100%|██████████| 1315/1315 [00:10<00:00, 126.93it/s]


## Submission

In [6]:
test_ids = ['0FB0700DAF44', '18409261F5C2',
            'D46BCB48440A', 'D72CB1C11673', 'DF920E0A7337']
txt_files = []
for i in test_ids:
    txt_file = open(
        f"../student_writing/data/test/{i}.txt", "r")
    txt_file = txt_file.read()
    txt_files.append(txt_file)


In [10]:
dfs = []
for j, item in enumerate(test_ids):
    entities = load_results_manual(txt_files[j], visualize=False)
    ids = [item for i in range(len(entities))]
    labels = [entities[i]['label'] for i in range(len(entities))]
    pred_string = []
    for i in range(len(entities)):
        start = int(entities[i]['start'])
        end = int(entities[i]['end'] + 1)
        pred_string.append(list(range(start, end)))
    test_dict = {'id': ids, 'class': labels, 'predictionstring': pred_string}
    df_test = pd.DataFrame.from_dict(test_dict)
    dfs.append(df_test)

submission_df = pd.concat(dfs)
submission_df = submission_df.reset_index().drop(['index'], axis=1)
submission_df.to_csv('./submission.csv')


In [8]:
submission_df


,id,class,predictionstring
0,0FB0700DAF44,LEAD,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
1,0FB0700DAF44,CLAIM,"[133, 134, 135, 136, 137, 138, 139, 140, 141, ..."
2,0FB0700DAF44,CLAIM,"[319, 320, 321, 322, 323, 324, 325, 326, 327, ..."
3,0FB0700DAF44,EVIDENCE,"[383, 384, 385, 386, 387, 388, 389, 390, 391, ..."
4,18409261F5C2,LEAD,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
5,18409261F5C2,EVIDENCE,"[62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 7..."
6,18409261F5C2,CLAIM,"[184, 185, 186, 187, 188, 189, 190, 191, 192, ..."
7,18409261F5C2,CLAIM,"[507, 508, 509, 510, 511, 512, 513, 514, 515, ..."
8,D46BCB48440A,LEAD,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
9,D46BCB48440A,CLAIM,"[167, 168, 169, 170, 171, 172, 173, 174, 175, ..."
